In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import tensorcircuit as tc
import jax
from jax import random
import tqdm
import numpy as np
import optax
import pandas as pd
import matplotlib.pyplot as plt
import time

from queso.io import IO
from queso import sensors
from queso.quantities import classical_fisher_information

backend = tc.set_backend("jax")
tc.set_dtype("complex128")
tc.set_contractor("auto")  # “auto”, “greedy”, “branch”, “plain”, “tng”, “custom”


2023-03-29 15:26:38.999702: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


functools.partial(<function custom at 0x12b5dd700>, optimizer=<function auto at 0x12a948e50>, memory_limit=None, debug_level=0)

In [3]:
folder = "estimator_test"
ansatz = "cnot_2local_ansatz"
n = 4
k = 4
seed = 1
repeat = 1

In [4]:
io = IO(folder=folder, include_date=False, include_id=False).subpath(ansatz)

lr = 0.20
progress = True
n_steps = 500
n_samples = 1000
fi_name = "qfi"

In [5]:
circ, shape = sensors.build(ansatz, n, k)

phi = 0.0
key = random.PRNGKey(seed)
theta = random.uniform(key, shape, minval=0, maxval=2*np.pi)

fi_val_grad_jit = backend.jit(
    backend.value_and_grad(
        lambda _theta: classical_fisher_information(circ=circ, theta=_theta, phi=phi, n=n, k=k),
        argnums=0,
    )
)

val, grad = fi_val_grad_jit(theta)
print(-val, -grad)

0.7287577094888074 [[-0.10375716 -0.59163989 -0.12732998  0.30253469  0.01242478]
 [-0.07654092  0.04961562 -0.20184303  0.46550994 -1.10277685]
 [ 0.01192723 -0.01859438 -0.38403487  0.30575809  0.08494393]
 [-0.25425551  0.17053121  0.64594925  0.20239222  1.00225256]
 [ 0.0219969   0.22238785  0.24831483 -0.21549646 -0.0591844 ]
 [-0.00587683  0.14134175 -0.31762331 -0.13362188 -0.088637  ]
 [ 0.72633955 -0.33995181  0.87221606  1.23151514  0.29664673]
 [ 0.82698389 -0.92570917  0.10498382 -0.22852398 -0.33127614]
 [-0.28353972  0.07724558 -0.49234412 -0.00700992 -0.30983054]
 [-0.56573128 -0.38612214 -0.15217872 -0.22115283 -0.26210316]
 [ 0.54404572 -0.38462052  0.11134509  0.01106845  0.33623389]
 [-0.08218624  0.38079681 -0.46240615  0.0422268   0.1772351 ]]


In [6]:
def _optimize(n_steps=250, lr=0.25, progress=True, subkey=None):
    opt = tc.backend.optimizer(optax.adagrad(learning_rate=lr))
    theta = random.uniform(subkey, shape, minval=0, maxval=2*np.pi)
    loss = []
    t0 = time.time()
    for step in (pbar := tqdm.tqdm(range(n_steps), disable=(not progress))):
        val, grad = fi_val_grad_jit(theta)
        theta = opt.update(grad, theta)
        loss.append(val)
        if progress:
            pbar.set_description(f"Cost: {-val:.10f}")
    t = time.time() - t0
    return -val, -np.array(loss), theta

In [7]:
df = []
print(f"\nOptimizing circuit: n={n}, k={k}")
key, subkey = random.split(key)
val, loss, theta = _optimize(
    n_steps=n_steps, lr=lr, progress=progress, subkey=subkey
)

# io.save_dataframe(pd.DataFrame(df), filename=f"optimization/n={n}_k={k}")
plt.pause(0.01)
print(phi)


Optimizing circuit: n=4, k=4


Cost: 15.9989431525: 100%|██████████| 500/500 [00:02<00:00, 227.08it/s]


0.0


In [8]:
fi_val_jit = backend.jit(
    lambda _theta, _phi: classical_fisher_information(circ=circ, theta=_theta, phi=_phi, n=n, k=k)
)
print(fi_val_grad_jit(theta))

(Array(-15.99894854, dtype=float64), Array([[-7.86608789e-04,  1.47185554e-03, -9.47434261e-03,
         5.99663945e-04,  6.06716760e-04],
       [ 2.12038107e-05, -1.75862418e-04,  8.80604983e-03,
        -3.60053890e-04, -9.57215510e-04],
       [-8.76945381e-05, -1.03780604e-03,  6.51710261e-04,
         4.57008407e-04,  1.67819106e-05],
       [-1.03747678e-03, -1.12040855e-04, -8.09763122e-04,
         1.55785843e-04,  3.74322815e-04],
       [-1.29935369e-03, -1.64016914e-03, -1.02699131e-03,
        -1.54627276e-04,  1.23135294e-03],
       [-2.08057083e-03,  4.07528977e-04, -2.18029500e-04,
         2.04860975e-04,  2.40060476e-06],
       [ 7.18841544e-04,  7.46976055e-04, -4.72252323e-03,
        -4.23687658e-04,  1.07128944e-05],
       [ 3.85086188e-04, -3.43275475e-03, -6.08432604e-04,
        -8.28448139e-04,  3.69366965e-05],
       [-5.95510842e-04, -1.03743984e-02,  1.23342115e-03,
        -7.58518755e-05,  9.36943486e-06],
       [ 1.43720889e-03, -1.74301482e-03, -4.

In [10]:
for phi in np.linspace(0, 2*np.pi, 100):
    fi = fi_val_jit(theta, phi)
    print(fi)

-15.998948540814478
-15.998946490731498
-15.998934994083172
-15.998908349921402
-15.998846735663664
-15.998666273332983
-15.99765023479378
-14.060359908731455
-15.997834416746601
-15.998671214059575
-15.99882328700022
-15.998866511291363
-15.998872680308995
-15.998856280377263
-15.998815505049595
-15.9987342978898
-15.998563009640483
-15.998119595143335
-15.996287624396427
-15.946076445660108
-15.989165434687612
-15.997349169175664
-15.998309081158103
-15.998587601291709
-15.998689963897993
-15.998718671762235
-15.99869839146675
-15.998622702783674
-15.99844770133181
-15.998017116316424
-15.996571367815092
-15.984110382600132
-15.931612994381647
-15.995146117399145
-15.997703068451528
-15.998318618537176
-15.998543984230855
-15.998633727021234
-15.998653316051032
-15.998614970994067
-15.998496043435388
-15.998203677216162
-15.997352637948586
-15.99300741490352
-13.89134598066339
-15.99115155960569
-15.997123988816393
-15.998137624840197
-15.99847055558576
-15.99860546409591
-15.9986533

In [42]:
n_shots = 1000
n_phi = 50
phis = np.linspace(0, np.pi, n_phi)
shots = np.zeros([n_shots, n_phi, n]).astype("int8")
probs = np.zeros([n_phi, 2**n])

In [43]:
for i, phi in enumerate(phis):
    # p = circ(theta, phi, n, k).probability()
    # probs[i, :] = p
    x, p = zip(*circ(theta, phi, n, k).sample(batch=n_shots, allow_state=True))
    shots[:, i, :] = np.stack(x).astype("int8")
# print(probs)

[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.

In [45]:
print(shots.shape)

(1000, 50, 4)


In [14]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
np.set_printoptions(precision=3, suppress=True)

from keras.models import Sequential
from keras.layers import Dense, Input
from keras import optimizers
from keras.preprocessing import sequence
from keras.utils import np_utils

In [46]:
# pre-process data
# X = shots.reshape([shots.shape[0] * shots.shape[1], shots.shape[2]])
X = shots
Y = np.repeat(phis, n_shots)

print(X.shape)
print(Y.shape)

(1000, 50, 4)
(50000,)


In [31]:
X = probs
Y = phis

print(X.shape)
print(Y.shape)

(200, 16)
(200,)


In [32]:
# define model
model = Sequential()
# model.add(Input(shape=(n,)))
model.add(Dense(100, input_dim=X.shape[1], activation="elu"))
model.add(Dense(100, activation="elu"))
model.add(Dense(100, activation="elu"))
model.add(Dense(1))
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_8 (Dense)             (None, 100)               1700      
                                                                 
 dense_9 (Dense)             (None, 100)               10100     
                                                                 
 dense_10 (Dense)            (None, 100)               10100     
                                                                 
 dense_11 (Dense)            (None, 1)                 101       
                                                                 
Total params: 22,001
Trainable params: 22,001
Non-trainable params: 0
_________________________________________________________________


In [40]:
opt = optimizers.Adam(learning_rate=0.00005)
model.compile(loss='mse', optimizer=opt)
model.fit(X, Y, epochs=500, verbose=2, batch_size=10)
# scores = model.evaluate(X, Y, verbose=0)
# print(scores)

Epoch 1/500
20/20 - 0s - loss: 0.0021 - 384ms/epoch - 19ms/step
Epoch 2/500
20/20 - 0s - loss: 1.9628e-04 - 27ms/epoch - 1ms/step
Epoch 3/500
20/20 - 0s - loss: 9.7578e-05 - 25ms/epoch - 1ms/step
Epoch 4/500
20/20 - 0s - loss: 6.1579e-05 - 22ms/epoch - 1ms/step
Epoch 5/500
20/20 - 0s - loss: 4.7850e-05 - 26ms/epoch - 1ms/step
Epoch 6/500
20/20 - 0s - loss: 4.7690e-05 - 25ms/epoch - 1ms/step
Epoch 7/500
20/20 - 0s - loss: 3.1839e-05 - 29ms/epoch - 1ms/step
Epoch 8/500
20/20 - 0s - loss: 4.2808e-05 - 27ms/epoch - 1ms/step
Epoch 9/500
20/20 - 0s - loss: 3.9629e-05 - 25ms/epoch - 1ms/step
Epoch 10/500
20/20 - 0s - loss: 3.5802e-05 - 26ms/epoch - 1ms/step
Epoch 11/500
20/20 - 0s - loss: 5.2589e-05 - 30ms/epoch - 2ms/step
Epoch 12/500
20/20 - 0s - loss: 4.2445e-05 - 39ms/epoch - 2ms/step
Epoch 13/500
20/20 - 0s - loss: 9.1768e-05 - 45ms/epoch - 2ms/step
Epoch 14/500
20/20 - 0s - loss: 1.2469e-04 - 43ms/epoch - 2ms/step
Epoch 15/500
20/20 - 0s - loss: 1.6685e-04 - 38ms/epoch - 2ms/step
Epoch 

In [36]:
Y_pred = model.predict(X).squeeze()
var = np.mean(np.abs(Y - Y_pred))
print(var)

7/7 [==============================] - 0s 2ms/step
0.01885147867114866


In [41]:
Y_pred = model.predict(X).squeeze()
print(Y_pred.shape)
print(np.stack([Y, Y_pred], axis=1))

7/7 [==============================] - 0s 1ms/step
(200,)
[[0.    0.012]
 [0.016 0.023]
 [0.032 0.035]
 [0.047 0.046]
 [0.063 0.059]
 [0.079 0.072]
 [0.095 0.085]
 [0.111 0.099]
 [0.126 0.114]
 [0.142 0.129]
 [0.158 0.144]
 [0.174 0.16 ]
 [0.189 0.176]
 [0.205 0.192]
 [0.221 0.208]
 [0.237 0.224]
 [0.253 0.24 ]
 [0.268 0.257]
 [0.284 0.273]
 [0.3   0.29 ]
 [0.316 0.306]
 [0.332 0.322]
 [0.347 0.339]
 [0.363 0.355]
 [0.379 0.372]
 [0.395 0.388]
 [0.41  0.405]
 [0.426 0.421]
 [0.442 0.437]
 [0.458 0.454]
 [0.474 0.47 ]
 [0.489 0.486]
 [0.505 0.502]
 [0.521 0.518]
 [0.537 0.534]
 [0.553 0.55 ]
 [0.568 0.566]
 [0.584 0.582]
 [0.6   0.598]
 [0.616 0.613]
 [0.631 0.629]
 [0.647 0.645]
 [0.663 0.66 ]
 [0.679 0.675]
 [0.695 0.691]
 [0.71  0.706]
 [0.726 0.721]
 [0.742 0.736]
 [0.758 0.751]
 [0.774 0.766]
 [0.789 0.782]
 [0.805 0.798]
 [0.821 0.814]
 [0.837 0.83 ]
 [0.852 0.846]
 [0.868 0.863]
 [0.884 0.879]
 [0.9   0.896]
 [0.916 0.912]
 [0.931 0.928]
 [0.947 0.944]
 [0.963 0.96 ]
 [0.979 0.97